# CC3001 Otoño 2020 Tarea 5
# Simulación con Colas y Colas de Prioridad

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Alumno: Matías Seda

Plazo: Miércoles 24 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender a escribir programas de simulación sencillos, haciendo uso de colas y de colas de prioridad.

En primer lugar, consideremos una caja a la que van llegando clientes en instantes aleatorios, separados por intervalos de tamaño promedio $t_a$. Cada cliente que llega se pone al final de la cola, espera su turno, y cuando llega a la cabeza de la cola lo empiezan a atender. La atención dura también un tiempo aleatorio de tamaño promedio $t_s$. Cuando completa su atención, sale del sistema, y en la caja se empieza a atender al siguiente cliente de la cola, en caso de que haya alguien.

![colaT5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/colaT5.png?raw=1)

Los tiempos entre llegadas y el tiempo de atención se modelan con distribuciones exponenciales, lo cual es la suposición usual en este tipo de simulaciones.

A continuación veremos un programa que simula este proceso para un número dado de clientes y calcula el largo promedio de la cola y la demora total promedio de los clientes.

El programa lleva un reloj simulado, que comienza en cero y va  avanzando monótonamente, al siguiente instante en que un cliente llega al sistema o al siguiente instante en que un cliente sale del sistema, lo que sea que ocurra primero.

Como hay solo dos tipos posible de eventos de interés, tendremos una variable para cada uno de ellos (``evento_llegada`` y ``evento_salida``), que almacenan los respectivos tiempos. Si en este momento no hay nadie que deba salir del sistema, ``evento_salida`` es ``None``, lo cual ocurre cada vez que la cola de espera queda vacía. En cambio, ``evento_llegada`` solo es ``None``cuando ya han llegado todos los clientes que debían llegar.

Nótese que no es necesario tener generados los tiempos de todas las llegadas futuras, basta con tener el tiempo de la próxima llegada, y cuando ella se cumple se genera la siguiente.

En cada elemento que está en la cola de espera anotamos solo la hora a la que llegó ese cliente, que es lo único que necesitamos saber para poder calcular, en el momento en que sale, cuánto tiempo demoró en el sistema.

Estudie con atención este programa y la bitácora que él genera.

In [1]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola()
    evento_llegada=np.random.exponential(ta)
    evento_salida=None
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    while evento_llegada is not None or evento_salida is not None:
        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if (evento_salida is None) or (evento_llegada is not None and (evento_llegada<=evento_salida)):
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            ahora=evento_llegada
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(ahora) # Basta anotar su hora de llegada
            acum_largo_cola+=c.size()
            
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
            
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                evento_llegada=ahora+np.random.exponential(ta)
            else:
                evento_llegada=None
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=evento_salida
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            acum_tiempo_en_sistema+=(ahora-tllegada)
            
            if c.is_empty(): # no hay nadie en la cola
                evento_salida=None
            else: # Hay un cliente esperando, empezamos a atenderlo
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)            

In [2]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [3]:
print_list(bitacora)

T=21.25986576184801 evento_llegada=21.25986576184801 evento_salida=None => LLEGADA
	cola=[]
T=78.83677538497456 evento_llegada=78.83677538497456 evento_salida=94.24603167524747 => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 evento_llegada=232.71542283099382 evento_salida=94.24603167524747 => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 evento_llegada=232.71542283099382 evento_salida=207.7973648436669 => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 evento_llegada=232.71542283099382 evento_salida=None => LLEGADA
	cola=[]
T=256.5855663702635 evento_llegada=265.07595569718336 evento_salida=256.5855663702635 => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 evento_llegada=265.07595569718336 evento_salida=None => LLEGADA
	cola=[]
T=386.489172340086 evento_llegada=582.4168679105603 evento_salida=386.489172340086 => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 evento_llegada=582.4168679105603 evento_salida=None => LLEGADA
	cola=[]


### 1. Mejorar este programa introduciendo una cola de prioridad

El programa está escrito sabiendo que puede haber dos tipos de eventos por ocurrir (LLEGADA y SALIDA) y que en todo momento puede haber a lo más solo uno de cada tipo.

En una situación más general, podría haber eventos de muchos tipos programados para ocurrir, en muchos instantes distintos. En ese caso, la implementación presentada es demasiado limitada, y se requiere un mecanismo más general para administrar esa lista de eventos futuros. Una *cola de prioridad* (usando el tiempo como prioridad, y en que el mínimo tiempo equivale a mejor prioridad) es la estructura adecuada, porque en esa lista de eventos futuros siempre vamos extrayendo el primero que debe ocurrir (el mínimo), y durante la ejecución se pueden generar nuevos elementos programados para ocurrir en el futuro (inserciones).

En preparación a poder simular modelos más complejos, lo que usted tiene que hacer es implementar una lista de eventos futuros mediante uan cola de prioridad, en la cual se almacenen pares de la forma

$$
(\text{tiempo},\text{tipo de evento})
$$

y modificar el programa para que utilice esa cola de prioridad en lugar de las variables ``evento_salida`` y ``evento_llegada``. Implemente la cola de prioridad en base al código para heaps que aparece en el apunte.

Sugerencias: Antes del while, encolar el primer evento de llegada, luego en cada iteración sacar un evento de la cola de prioridad y procesarlo, y cuando en el programa original se generaba un evento nuevo y se asignaba a la variable respectiva, ahora hay que insertar un nuevo evento del tipo correspondiente en la cola de prioridad.

El programa resultante debiera ser más simple, porque varias cosas se unifican al verlas de esta manera.

Ejecute ambos programas en modo ``debug`` y asegúrese que en las bitácoras resultantes los eventos que ocurren y sus tiempos sean idénticos.

## Lo que usted tiene que hacer

## 2. Modificar el modelo para introducir un tiempo comprando

A continuación, introduciremos un elemento adicional en el modelo, y usted debe modificar su programa para incluirlo.

Supondremos que una vez que un cliente llega, en lugar de ponerse a la cola de inmediato, pasa un rato en la tienda comprando, y una vez que termina (después de transcurrido un tiempo aleatorio de duración promedio $t_c$), en ese momento recién se pone a la cola para pagar en la caja.

![cola2T5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/cola2T5.png?raw=1)

Observe que, como puede haber muchos clientes en la tienda que aún no terminan de comprar, en la lista de eventos futuros ahora puede haber muchos eventos de un nuevo tipo (FINCOMPRA), lo que justifica el haber introducido la cola de prioridad.

Ejecute su programa en modo ``debug`` agregando un parámetro $t_c=300$ e imprima el largo promedio de la cola, la demora total promedio de los clientes y la bitácora.

## 3. ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Haga todos los cambios necesarios para explicar y documentar adecuadamente su código. No olvide poner su nombre.

# Entrega

### 1. Simulación con cola de prioridad
El siguiente código presente la implementación de la simulación anterior ocupando una cola de prioridad.

En primera instancia, se define lo necesario para implementar la cola de prioridad.

In [4]:
class Evento:
    def __init__(self,tipo,tiempo):
        self.tiempo = tiempo
        self.tipo = tipo

def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j].tiempo < a[(j-1)//2].tiempo:
        (a[j],a[(j-1)//2])=(a[(j-1)//2],a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre

        
def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1].tiempo <a[k].tiempo : # el hijo derecho existe y es mayor
            k+=1
        if a[j].tiempo<=a[k].tiempo: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el mayor de los hijos
        j=k # bajamos al lugar del mayor de los hijos
    
class Heap:
    def __init__(self,maxn=100):
        self.a=np.ndarray(maxn,dtype = object)
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x
        trepar(self.a,self.n)
        self.n+=1
    def extract_min(self):
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def imprimir(self):
        print(self.a[0:self.n])


Más específicamente, primero se define una clase ```Evento``` que almacena la información del tipo del evento y su hora de término.

```python
class Evento:
    def __init__(self,tipo,tiempo):
        self.tiempo = tiempo
        self.tipo = tipo
```

Luego, se define las funciones necesaria para manipular el Heap.

```python
def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j].tiempo < a[(j-1)//2].tiempo:
        (a[j],a[(j-1)//2])=(a[(j-1)//2],a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre

        
def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1].tiempo <a[k].tiempo : # el hijo derecho existe y es mayor
            k+=1
        if a[j].tiempo<=a[k].tiempo: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el mayor de los hijos
        j=k # bajamos al lugar del mayor de los hijos
```
Nótese que estas funciones consideran que la lista ```a``` almacena objetos de la clase ```Evento``` y ordenan listas de objetos de la clase ```Evento``` respecto a sus atributos ```tiempo``` tal que la lista ```a``` representa un heap donde el elemento con mayor prioridad es el mínimo.

Finalmente, se define la clase ```Heap``` con sus respectivos métodos.

```python
class Heap:
    def __init__(self,maxn=100):
        self.a=np.ndarray(maxn,dtype = object)
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x
        trepar(self.a,self.n)
        self.n+=1
    def extract_min(self):
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
```
Nótese que al inicializar un objeto de la clase, se establece que el arreglo asociado al objeto puede almacenar otros objetos (como objetos de la clase ```Evento```).

Ahora, respecto a la implementación de la simulación con la cola de prioridad, el código es el siguiente:

In [5]:
def simulaHeap(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)

    ahora=0 # tiempo simulado
    nclientes = 0# número de clientes que han llegado
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug

    #Se crea la cola de prioridad (heap)
    heap=Heap(maxclientes)
    
    #Se crea una cola para simular la fila de la cola y para guardar el tiempo de llegada de cada cliente y poder calcular el tiempo que el cliente está en el sistema y, así, calcular el promedio al final.
    cola = Cola()

    #Se inserta el primer elemento (evento) al heap 
    evento = Evento("Llegada", np.random.exponential(ta))
    heap.insert(evento)
    
    #La simulación ocurre mientras la cola de prioridad no esté vacía (que esté vacía significa que ya ocurrieron todos los eventos).
    while heap.n !=0:
        #Se extraer el evento con tiempo mínimo, es decir, el evento que debe ocurrir
        evento = heap.extract_min()
        ahora = evento.tiempo

        if debug:
            s = "T = " + str(ahora) + " ===> " + evento.tipo
            bitacora.append(s)

        #Si el evento es del tipo Llegada
        if evento.tipo == "Llegada":
            #Se el cliente se pone en la cola, guardando su tiempo de llegada 
            cola.enq(evento.tiempo)
            acum_largo_cola += cola.size()
            nclientes += 1

            #Si el cliente es el único en la cola, se le atiende
            if cola.size() == 1:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))

            #Si existen más clientes por llegar, se establece la hora de llegada del próximo cliente
            if nclientes < maxclientes:
                heap.insert(Evento("Llegada", ahora + np.random.exponential(ta)))
                
        #Si el evento es del tipo Salida
        else:
            #El cliente sale de la cola
            tllegada = cola.deq()
            #Se calcula su tiempo total en el sistema y se guarda en un total acumulado de tiempos de los clientes en el sistema
            acum_tiempo_en_sistema += (ahora-tllegada)

            #Si en la cola quedan más clientes, se programa la salida del próximo cliente
            if cola.size() !=0:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))

    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)


En el código se encuentran comentarios explicando la lógica del código, sin embargo, el términos generales el codigo realiza lo siguiente:

Se inicializa la función, en caso de tener debug se genera bitácora y usa secuencia aleatoria reproducible y se inicializan los objetos necesarios para realizar la simulación:

```python
def simulaHeap(maxclientes,ta,ts,debug=False):
    if debug:
        np.random.seed(1234)

    ahora=0 
    nclientes = 0
    acum_largo_cola=0 
    acum_tiempo_en_sistema=0 
    bitacora=[] 

    heap=Heap(maxclientes)
    
    cola = Cola()
```

Se agrega el primer evento al heap y se entra al ciclo while (dado que existe un evento por ocurrir):

```python
    evento = Evento("Llegada", np.random.exponential(ta))
    heap.insert(evento)
    
    while heap.n !=0:
```

Dentro del ciclo while, se extrae el evento con tiempo mínimo, es decir, el evento que debe ocurrir y en caso de tener debug, se agrega el evento a la bitácora.
```python
        evento = heap.extract_min()
        ahora = evento.tiempo

        if debug:
            s = "T = " + str(ahora) + " ===> " + evento.tipo
            bitacora.append(s)
```

Ahora, el evento puede ser un evento de llegada o de salida. 

Si el  evento es un evento de llegada, el cliente se pone a la fila. Además, si en la fila hay solo un cliente, se atiende al ciente y si quedan por llegar más clientes, se asigna la hora de llegada del próximo cliente.
```python 
        if evento.tipo == "Llegada": 
            cola.enq(evento.tiempo)
            acum_largo_cola += cola.size()
            nclientes += 1

            if cola.size() == 1:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))

            if nclientes < maxclientes:
                heap.insert(Evento("Llegada", ahora + np.random.exponential(ta)))
```
Si el evento es del tipo salida, el cliente  sale de la fila y termina su paso por el sistema. En caso de que la fila no esté vacía, se programa la salida del próximo cliente.

```python
        else:
            tllegada = cola.deq()
            acum_tiempo_en_sistema += (ahora-tllegada)
            
            if cola.size() !=0:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))
```

En caso de no quedar más eventos por ocurrir, se finaliza la simulación y se retornan el largo promedio de la fila, el tiempo promedio en el sistema y la bitácora.

```python
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)
```
Ahora, en el siguiente ejemplo se prueba la simulación y se genera e imprime su bitácora respectiva.

In [6]:
(cola,demora,bitacora)=simulaHeap(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)
print_list(bitacora)


Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722
T = 21.25986576184801 ===> Llegada
T = 78.83677538497456 ===> Llegada
T = 94.24603167524747 ===> Salida
T = 207.7973648436669 ===> Salida
T = 232.71542283099382 ===> Llegada
T = 256.5855663702635 ===> Salida
T = 265.07595569718336 ===> Llegada
T = 386.489172340086 ===> Salida
T = 582.4168679105603 ===> Llegada
T = 626.7051068662289 ===> Llegada
T = 696.2190483450556 ===> Llegada
T = 738.9366623619304 ===> Salida
T = 811.2505419244258 ===> Llegada
T = 832.4793149848136 ===> Salida
T = 857.4938981550002 ===> Llegada
T = 894.2570294009206 ===> Salida
T = 895.296837936396 ===> Salida
T = 927.427158306087 ===> Llegada
T = 1006.4499708056504 ===> Salida
T = 1167.1389175826548 ===> Salida


### 2. Simulación con cola de prioridad y un tiempo comprando

El siguiente código presente la implementación de la simulación utilizando una cola de prioridad y agregando un tiempo de compra. 

En primera instancia, se necesita agregar un parámetro a la clase ```Evento``` y, por lo tanto, se redefine la clase.

In [7]:
class Evento:
    def __init__(self,tipo,tiempo,tiempo2=0):
        self.tiempo = tiempo
        self.tiempo2 = tiempo2
        self.tipo = tipo

La necesidad de agregar el parámetro ```tiempo2``` a la clase ```Evento``` es debido a que, cuando se tenga un evento del tipo fin de compra, se necesita almacenar el tiempo de llegada del cliente asociado al evento.

Ahora, respecto a la implementación de la simulación con la cola de prioridad y con un tiempo de compra, el código es el siguiente:

In [8]:
def simulaHeapCompra(maxclientes,ta,ts,tc,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # tc = tiempo promedio de compra
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)

    ahora=0 # tiempo simulado
    nclientes = 0# número de clientes que han llegado
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    #Se crea la cola de prioridad (heap)
    heap=Heap(maxclientes)
    
    #Se crea una cola para simular la fila de la cola y para guardar el tiempo de llegada de cada cliente y poder calcular el tiempo que el cliente está en el sistema y, así, calcular el promedio al final.
    cola = Cola()

    #Se inserta el primer elemento (evento) al heap 
    evento = Evento("Llegada", np.random.exponential(ta))
    heap.insert(evento)

    #La simulación ocurre mientras la cola de prioridad no esté vacía (que esté vacía significa que ya ocurrieron todos los eventos).
    while heap.n !=0:
        #Se extraer el evento con tiempo mínimo, es decir, el evento que debe ocurrir
        evento = heap.extract_min()
        ahora = evento.tiempo

        if debug:
            s = "T = " + str(ahora) + " ===> " + evento.tipo
            bitacora.append(s)
            
        #Si el evento es del tipo Llegada
        if evento.tipo == "Llegada":
            #El cliente llega y va a comprar. Se genera un evento del tipo FinCompra y se guarda en el heap. Este evento tiene un segundo tiempo que almacena el tiempo de llegada del cliente.
            heap.insert(Evento("FinCompra", ahora + np.random.exponential(tc), ahora))
            nclientes += 1

            #Si existen más clientes por llegar, se establece la hora de llegada del próximo cliente
            if nclientes < maxclientes:
                heap.insert(Evento("Llegada", ahora + np.random.exponential(ta)))

        #Si el evento es del tipo FinCompra
        elif evento.tipo == "FinCompra":
            #El cliente deja de comprar y se pone en la cola, guardando su tiempo de llegada
            cola.enq(evento.tiempo2)
            acum_largo_cola += cola.size()

            #Si el cliente es el único en la cola, se le atiende
            if cola.size() == 1:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))
                
        #Si el evento es del tipo Salida
        else:
            #El cliente sale de la cola
            tllegada = cola.deq()
            #Se calcula su tiempo total en el sistema y se guarda en un total acumulado de tiempos de los clientes en el sistema
            acum_tiempo_en_sistema += (ahora-tllegada)

            #Si en la cola quedan más clientes, se programa la salida del próximo cliente
            if cola.size() !=0:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))

    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)

En el código se encuentran comentarios explicando la lógica del código, sin embargo, el términos generales el codigo realiza lo siguiente:

Se inicializa la función, en caso de tener debug se genera bitácora y se usa una secuencia aleatoria reproducible y se inicializan los objetos necesarios para realizar la simulación:

```python
def simulaHeap(maxclientes,ta,ts,debug=False):
    if debug:
        np.random.seed(1234)

    ahora=0 
    nclientes = 0
    acum_largo_cola=0 
    acum_tiempo_en_sistema=0 
    bitacora=[] 

    heap=Heap(maxclientes)
    
    cola = Cola()
```

Se agrega el primer evento al heap y se entra al ciclo while (dado que existe un evento por ocurrir):

```python
    evento = Evento("Llegada", np.random.exponential(ta))
    heap.insert(evento)
    
    while heap.n !=0:
```

Dentro del ciclo while, se extrae el evento con tiempo mínimo, es decir, el evento que debe ocurrir y en caso de tener debug, se agrega el evento a la bitácora.
```python
        evento = heap.extract_min()
        ahora = evento.tiempo

        if debug:
            s = "T = " + str(ahora) + " ===> " + evento.tipo
            bitacora.append(s)
```
Ahora, el evento puede ser un evento de llegada, evento de fin de compra o un evento de salida. 

Si el  evento es del tipo de llegada, el cliente empieza a comprar y, por tanto, se genera un evento fin de compra que guarda, asimismo, el tiempo de llegada del cliente. Además, si quedan clientes por llegar, se asigna la hora de llegada del próximo cliente. 
```python 
        if evento.tipo == "Llegada":
            heap.insert(Evento("FinCompra", ahora + np.random.exponential(tc), ahora))
            nclientes += 1

            if nclientes < maxclientes:
                heap.insert(Evento("Llegada", ahora + np.random.exponential(ta)))
```
Si el  evento es del tipo de fin de compra, el cliente se pone a la fila y si el client es el único en la fila, se le atiende.
```python
        elif evento.tipo == "FinCompra":
            cola.enq(evento.tiempo2)
            acum_largo_cola += cola.size()
            
            if cola.size() == 1:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))
```

Si el evento es del tipo salida, el cliente  sale de la fila y termina su paso por el sistema. En caso de que la fila no esté vacía, se programa la salida del próximo cliente.

```python
        else:
            tllegada = cola.deq()
            acum_tiempo_en_sistema += (ahora-tllegada)
            
            if cola.size() !=0:
                heap.insert(Evento("Salida", ahora + np.random.exponential(ts)))
```

En caso de no quedar más eventos por ocurrir, se finaliza la simulación y se retornan el largo promedio de la fila, el tiempo promedio en el sistema y la bitácora.

```python
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)
```
Ahora, en el siguiente ejemplo se prueba la simulación y se genera e imprime su bitácora respectiva.

In [9]:
(cola,demora,bitacora)=simulaHeapCompra(10,100,75,300,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)
print_list(bitacora)


Largo promedio de la cola= 1.5  Tiempo promedio en el sistema= 417.5468811785192
T = 21.25986576184801 ===> Llegada
T = 78.83677538497456 ===> Llegada
T = 230.23855294286716 ===> Llegada
T = 262.5990858090567 ===> Llegada
T = 313.20452941544585 ===> FinCompra
T = 325.719127099946 ===> FinCompra
T = 469.724323866816 ===> Salida
T = 502.9405030835674 ===> Salida
T = 540.4727177230324 ===> FinCompra
T = 579.9399980224337 ===> Llegada
T = 592.6081738321524 ===> Salida
T = 704.6635348529446 ===> Llegada
T = 748.2519523806671 ===> FinCompra
T = 787.0338207410872 ===> Llegada
T = 791.1930548829886 ===> FinCompra
T = 800.7018974939823 ===> Salida
T = 843.3936035446682 ===> FinCompra
T = 925.0344787605443 ===> FinCompra
T = 935.2379979000931 ===> Llegada
T = 961.3908442709866 ===> Salida
T = 967.268877737664 ===> Salida
T = 1001.7816684248819 ===> Salida
T = 1030.792148840738 ===> Llegada
T = 1071.4232208888677 ===> FinCompra
T = 1109.3862775258685 ===> Salida
T = 1136.1688937550139 ===> Llegad

### Discusiones y conclusiones

1. Lo más relevante a destacar es que el uso de una cola de prioridad permitió implementar un código más sencillo y conciso. La utilización de las variables ```evento_llegada``` y ```evento_salida``` generaban un código engorroso y, del mismo modo, agregar otro tipo de eventos presentaba dificultades. Con la cola de prioridad, el código se tornó más simple y el agregar un nuevo tipo de evento no fue una tarea complicada.


2. Respecto a la simulación con cola de prioridad y evento de fin de compra, es coherente que el tiempo promedio en el sistema sea de 417.5468811785192, dado que ```ts```(tiempo promedio de servicio) es $75$ y ```tc```(tiempo promedio de compra) es $300$. En ese sentido, el promedio en el sistema debería que ser $375$ pero dado, que se está utilizando una distribución exponencial para calcular el tiempo de servcio y el tiempo de comprar para cada cliente, el resultado no es exacatamente $375$ pero con alta probabilidad ronda en valores cercano a éste. Así, el valor obtenido $417.5468811785192$ es coherente.


3. Del mismo modo, la bitácora presenta resultados coherentes con los valores de ```ta```, ```ts```, ```tc```. Por ejemplo, los eventos del inicio son
```
T = 21.25986576184801 ===> Llegada
T = 78.83677538497456 ===> Llegada
T = 230.23855294286716 ===> Llegada
T = 262.5990858090567 ===> Llegada
T = 313.20452941544585 ===> FinCompra
```
Dado ```ta``` es $100$ y ```tc``` es $300$, es más probable (dado que se utiliza una distribución exponencial) que, cuando el primer cliente llega y empieza su compra, lleguen más clientes antes que el primer cliente termine su compra. En este caso, luego de que llega el primer cliente y comienza su compra, llegan tres clientes antes de que el primer cliente termine su compra, lo cual es bastante coherente dado que ```ta``` es $100$ y ```tc``` es $300$ (puesto que, en promedio, demora $300$ en comprar pero un cliente, en promedio, llega cada $100$, es decir, en promedio, llegan 3 clientes antes de que un cliente termine sus compras.)

Así, finalmente, el código genera un resultado esperado dado los valores entregados.
